In [3]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np 
#for importing files
import glob, os
import matplotlib.pyplot as plt
%matplotlib inline


#Library for hull calculation
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
from matplotlib.path import Path
from scipy.spatial import distance


## Helper Methods

In [4]:
def SecsToGameTime(time):
    mins=(int(time)/60)
    secs=int(time)%60
    print (str(mins)+":"+str(secs))

In [5]:
def GameTimetoSeconds(time):
    mins, secs = time.split(":")
    timeSecs = int(mins)*60+int(secs)
    print(timeSecs)
    return timeSecs

#### Hull Calculation Helper Method

In [6]:
def getHull(moments, teamID):
    points = []
    
    if(int(moments[5][1][0]) == teamID):
        for positions in moments[5][1:6]:
            #print positions
            points.append([positions[2],positions[3]])
        print(points)
    else:
        for positions in moments[5][6:11]:
            #print positions
            points.append([positions[2],positions[3]])
        print(points)
    
    np_points = np.asarray(points)
    np_points
    hull = ConvexHull(np_points)
    
    return hull,np_points

In [7]:
def distFromCentroid(hull, x,y):
        cx = np.mean(hull.points[hull.vertices,0])
        cy = np.mean(hull.points[hull.vertices,1])
        return ((x-cx)^2+(y-cy)^2)^.5

In [8]:
def isInHull(hull,np_points,x,y):
    #point in hull?
    hull_path = Path( np_points[hull.vertices] )
    isInPath = hull_path.contains_point((x,y))
    return (isInPath == True)

### Helper method for returning PBP descriptions given game and eventNum

In [19]:
def getPBPInfo(gameID,PBPEventNum):
    curPath = os.getcwd()
    if(not curPath.endswith('\pbp-csv')):
        curPath = os.getcwd()
        print curPath
        strIndex=curPath.index('\\JupyterNotebooks')
        curPath = curPath[:(strIndex+18)]+'\pbp-csv'
        os.chdir(curPath)
        print curPath
    
    events_df = pd.read_csv("00"+str(gameID)+".csv")
    eventRow = events_df[events_df['EVENTNUM'].isin([int(PBPEventNum)])]
     
    print eventRow['PCTIMESTRING']
    print eventRow['HOMEDESCRIPTION']
    print eventRow['VISITORDESCRIPTION']
    return eventRow['PCTIMESTRING'] , eventRow['HOMEDESCRIPTION'] , eventRow['VISITORDESCRIPTION]
    

### Given the game time (12:00) and eventNumber from the Play-by-play data for a recorded event, this method returns the correct eventNumber from the SportsVU data since they actually dont matchup.  This method converts the gametime into seconds (720s = 12:00) which is formatted in SportsVU.  It traverses up and down events until the gametime is within a selected eventNumber.

In [57]:
def findCorrectMomentData(rawSportVUData, quarterPBP, gametimePBP, approxEventNumPBP):
    print("Finding correct SportVU eventNum for gameid:"+rawSportVUData['gameid'])

    found = False;
    moments = None
    #convert gametimePBP (12:00) to seconds (720)
    gameInSeconds = GameTimetoSeconds(gametimePBP)
    while (found == False):
        print("Currently checking eventNumPBP: "+str(approxEventNumPBP))
        moments = MomentsfromEventNum(rawSportVUData, approxEventNumPBP)
        if (moments is None or len(moments) == 0):
            return None
        
        quarterSportVU = int(moments[0][0])
        #print("PBP quareter:"+str(quarterPBP)+"quartersportVU :"+str(quarterSportVU))
        #print("PBP gameclock:"+str(gameInSeconds)+" SportsVU moment clock:"+str(moments[0][2]))
        
        #check whether this EventNum moment is the correct time frame of the play from Play-by-play
        if(quarterPBP < quarterSportVU):
            approxEventNumPBP -= 1
        elif(quarterPBP > quarterSportVU):    
            approxEventNumPBP  += 1
        elif(int(moments[0][2]) > gameInSeconds and int(moments[len(moments)-1][2]) < gameInSeconds):
            found=True
        elif(moments[0][2] < gameInSeconds):
            approxEventNumPBP -= 1
        elif(moments[len(moments)-1][2] > gameInSeconds):
        # look ahead 1 event 
            approxEventNumPBP += 1
        else:
            return None
    #print("extracted event num "+ str(approxEventNumPBP))
    return approxEventNumPBP

In [55]:
def MomentsfromEventNum(eventsData, eventNumPBP):
    found = False
    searchDirection = None
    sportVUIndex = eventNumPBP
    eventsLen = len(eventsData['events'])-1
    if(eventsLen < sportVUIndex):
        sportVUIndex = eventsLen
    
    while(not found):        
        if( int(eventsData['events'][sportVUIndex]['eventId']) == eventNumPBP):
            found = True
        elif ( int(eventsData['events'][sportVUIndex]['eventId']) > eventNumPBP ):
            if(searchDirection is None and sportVUIndex > 0):
                searchDirection = "decrementIndex"
                sportVUIndex -= 1  
            elif(searchDirection is "decrementIndex" and sportVUIndex > 0):
                sportVUIndex -= 1  
            elif(searchDirection is "incrementIndex"):
                return None
            else: #we have reached 0th index and cant traverse up the list anymore
                return None
        elif ( int(eventsData['events'][sportVUIndex]['eventId']) < eventNumPBP ):
            if(searchDirection is None and sportVUIndex < eventsLen):
                searchDirection = "incrementIndex"
                sportVUIndex += 1  
            elif(searchDirection is "incrementIndex" and sportVUIndex < eventsLen):
                sportVUIndex += 1  
            elif(searchDirection is "decrementIndex"):
                return None
            else: #we have reached 0th index and cant traverse up the list anymore
                return None
    print("sportVU Index for the eventNumPBP: "+str(sportVUIndex))
    
    return (eventsData['events'][sportVUIndex]["moments"])

## Identify Moment of CATCH

1.  //sort events moment data from end time to beginning (reverse order)
timeballReceived, locationballReceived = getReceive (recieverID, List[Moments]) 

Each moment is taken every .25s, so the FIRST 8 moments (8 *.25 = 2 seconds) when the receiverID and ball are within (dist <= 1 ft), is when the ball was the moment before the ball was released.

reset numCount = 0;

Starting from this time, continue iterating through moments. If dist from receiver and ball >=1 ft, then this is time the ball was caught.

***separate calculation to check when the ball dist less than 1 ft for another teammate to determine if it was a pass in a certain amount of time (counted numCount)****

2.  Find "latest" instance when   distFrom ( Moment.Positions[0].x,Moment.Position[0].y , Moment.Position[ ??=playerID ].x , Moment.Position[ ??=playerID].y) <= 1ft )

   return the timemilis ,  position[0].x , position[0].y


In [31]:
 def getCatchEventDetails(receiverID, passerID, gametimePBP, moments):
    momentsReversed = moments[::-1]
    processComplete = False;
    indexBallCaught = None
    indexBallReleased = None
    finalIndex=0
    index=0
    found = False
    gameInSeconds = GameTimetoSeconds(gametimePBP)
    print("num of moments: "+str(len(momentsReversed)))
    while (found == False):
        if(int(momentsReversed[index][2]) >=  gameInSeconds):
            found=True
        elif(int(momentsReversed[index][2]< gameInSeconds)):
            index += 1
        else:
            return "ERROR"
    
    print("starting at time: "+ str(momentsReversed[index][2]))
    
    #Found the index of the approx start of the event
    numCounts=0
    for i in range(index,len(momentsReversed)):
        if numCounts >= 3:
            timeBallReleased = momentsReversed[i][2]
            SecsToGameTime(timeBallReleased)  
            indexBallReleased = i
            break
        else:
            ball = momentsReversed[i][5][0]
            for positions in momentsReversed[i][5]:
                if positions[1] == receiverID:
                    player_x = positions[2]
                    player_y = positions[3]
            dist = distance.euclidean([player_x,player_y],[ball[2],ball[3]])
            if dist <= 2:
                print("receiver holding ball: " + str( momentsReversed[i][3]))
                numCounts += 1
    
    #check if ball caught event in moments list
    if(indexBallReleased is None):            
        return None, None, None, None, None, None, None, None, None, None
    
    #count when ball is caught
    numCounts=0
    for j in range(indexBallReleased, len(momentsReversed)):
        if numCounts >= 3:
            timeBallCaught = momentsReversed[j][2]
            SecsToGameTime(timeBallCaught)
            indexBallCaught = j
            
            #record position of receiver
            receiverMoment = momentsReversed[j][5]
            receiverPos = [k for k in receiverMoment if receiverID in k]
            
            reciever_pos_x = receiverPos[0][2]
            receiver_pos_y = receiverPos[0][3]
            receiverPosAtCaught = [reciever_pos_x, receiver_pos_y]
            
            break
        else:
            ball = momentsReversed[j][5][0]
            for positions in momentsReversed[j][5]:
                if positions[1] == receiverID:
                    player_x = positions[2]
                    player_y = positions[3]
            dist = distance.euclidean([player_x,player_y],[ball[2],ball[3]])
            if dist >= 2:
                print("receiver just caught ball: " + str(momentsReversed[j][3])+" received reversed index"+str(j))
                SecsToGameTime(momentsReversed[j][3])
                numCounts += 1
            if(momentsReversed[j][3] is None):            
                return None, None, None, None, None, None, None, None, None, None   
            
    
    
    #check if ball caught event in moments list
    if(indexBallCaught is None):            
        return None, None, None, None, None, None, None, None, None, None   
                
                
    #count when ball is passed
    numCounts=0
    for k in range(indexBallCaught, len(momentsReversed)):
        if numCounts >= 2:
            print("player PASSED ball at "+str(momentsReversed[k][3]))
            timeBallPassed = momentsReversed[k][2]
            SecsToGameTime(timeBallPassed)
            indexBallPassed = k
            processComplete = True
               
            #record position of receiver
            receiverMoment = momentsReversed[k][5]
            receiverPos = [k for k in receiverMoment if receiverID in k]
            
            reciever_pos_x = receiverPos[0][2]
            receiver_pos_y = receiverPos[0][3]
            receiverPosAtPass = [reciever_pos_x ,receiver_pos_y]
        
            #record position of passer
            passerPos = [k for k in receiverMoment if passerID in k]
            
            passer_pos_x = passerPos[0][2]
            passer_pos_y = passerPos[0][3]
            passerPosAtPass = [passer_pos_x ,passer_pos_y]
            
            SportsVUReversedIndex = len(momentsReversed) - indexBallPassed
            
            break
        else:
            ball = momentsReversed[k][5][0]
            for positions in momentsReversed[k][5]:
                if positions[1] == passerID:
                    player_x = positions[2]
                    player_y = positions[3]
            dist = distance.euclidean([player_x,player_y],[ball[2],ball[3]])
            if dist <= 2:
                print("passer just passed ball: " +str(momentsReversed[k][3]) + " passed reversed index"+str(k))
                numCounts += 1
            
            
                
    
    if(not processComplete):            
        return None, None, None, None, None, None, None, None, None , None
    
        
    return timeBallReleased, indexBallReleased, timeBallCaught, indexBallCaught, timeBallPassed, indexBallPassed, receiverPosAtPass, receiverPosAtCaught, passerPosAtPass, SportsVUReversedIndex 
   

### Is receiving player open calculation

In [59]:
import math
# is_open
#   determines whether a player is "open" based on distance of nearest defender
#   note that the moment that the player shoots isn't determined in this function
# Params:
#   shooter_loc: location of the shooter at a given moment as a tuple (x, y)
#   def_locs: locations of each defender as list of tuples (x, y)
def is_open(shooter_loc, def_locs):

    dists = []
    for loc in def_locs:
        dist = math.hypot(shooter_loc[0] - loc[0], shooter_loc[1] - loc[1])
        if dist < 5:
            return dist, False
        dists.append(dist)
    dists = sorted(dists)
    return dists[0], True

shooter_loc = (45.7042, 15.57870)
def_locs = [(56.27694, 25.51460), (73.64107, 25.48774), (48.6492, 35.26915), (48.46352, 14.55436), (47.51449, 24.36448)]
print(is_open(shooter_loc, def_locs))

(2.943317736500769, False)


### Extracts Assists, Bad Passes, Alley Oops from csvs in current directory and saves them into separate csv files

In [49]:
AST_df = pd.DataFrame()
BadPass_df = pd.DataFrame()
AlleyOop_df = pd.DataFrame()
curPath = os.getcwd()
print curPath
if(not curPath.endswith('\pbp-csv')):
    curPath = os.getcwd()
    print curPath
    strIndex=curPath.index('\\JupyterNotebooks')
    curPath = curPath[:(strIndex+18)]+'\pbp-csv'
    os.chdir(curPath)
    print curPath
for pbpFile in glob.glob("*.csv"):
    print(pbpFile)
    #read file as df
    events_df = pd.read_csv(pbpFile)
    ASTList = events_df[events_df['EVENTMSGTYPE'].isin([1,2]) & ( events_df['VISITORDESCRIPTION'].str.contains("AST", na = False ) | events_df['HOMEDESCRIPTION'].str.contains("AST", na = False )  ) ]
    BadPassList= events_df[( events_df['VISITORDESCRIPTION'].str.contains("Bad", na = False ) | events_df['HOMEDESCRIPTION'].str.contains("Bad", na = False )  ) ]
    AlleyOopList= events_df[events_df['EVENTMSGTYPE'].isin([1,2]) & ( events_df['VISITORDESCRIPTION'].str.contains("Alley", na = False ) | events_df['HOMEDESCRIPTION'].str.contains("Alley", na = False )  ) ]
    AST_df = AST_df.append(ASTList)
    BadPass_df = BadPass_df.append(BadPassList)
    AlleyOop_df = AlleyOop_df.append(AlleyOopList)
    
AST_df.to_csv("AST_Parsed.csv", sep='\t')
BadPass_df.to_csv("BadPass_Parsed.csv", sep='\t')
AlleyOop_df.to_csv("AlleyOop_Parsed.csv", sep='\t')

D:\DataMining-NBA-Repo\JupyterNotebooks\SportVU_JSON_Data
D:\DataMining-NBA-Repo\JupyterNotebooks\SportVU_JSON_Data
D:\DataMining-NBA-Repo\JupyterNotebooks\\pbp-csv
0021500001.csv
0021500002.csv


## AST_df  assists data frame has time and event num of all assists....

In [14]:
AST_df

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION
5,21500001,5,1,80,1,8:13 PM,11:21,NaN,NaN,Marc Morris 13' Step Back Jump Shot (2 PTS) (D...,...,Detroit,Pistons,DET,0,0,NaN,NaN,NaN,NaN,NaN
6,21500001,6,1,80,1,8:13 PM,11:00,Millsap 12' Step Back Jump Shot (2 PTS) (Korve...,NaN,NaN,...,Atlanta,Hawks,ATL,0,0,NaN,NaN,NaN,NaN,NaN
7,21500001,7,1,101,1,8:14 PM,10:44,NaN,NaN,Caldwell-Pope 9' Driving Floating Jump Shot (2...,...,Detroit,Pistons,DET,0,0,NaN,NaN,NaN,NaN,NaN
12,21500001,13,1,105,1,8:14 PM,10:01,Millsap 12' Turnaround Fadeaway Bank Jump Shot...,NaN,NaN,...,Atlanta,Hawks,ATL,0,0,NaN,NaN,NaN,NaN,NaN
28,21500001,33,1,7,1,8:17 PM,8:11,NaN,NaN,Drummond Dunk (2 PTS) (Ilyasova 2 AST),...,Detroit,Pistons,DET,0,0,NaN,NaN,NaN,NaN,NaN
40,21500001,47,1,1,1,8:22 PM,6:48,NaN,NaN,Ilyasova 24' 3PT Jump Shot (3 PTS) (Marc Morri...,...,Detroit,Pistons,DET,0,0,NaN,NaN,NaN,NaN,NaN
46,21500001,55,1,63,1,8:23 PM,6:30,NaN,NaN,Marc Morris 20' Fadeaway Jumper (7 PTS) (Jacks...,...,Detroit,Pistons,DET,0,0,NaN,NaN,NaN,NaN,NaN
49,21500001,58,1,1,1,8:24 PM,5:57,NaN,NaN,Caldwell-Pope 24' 3PT Jump Shot (5 PTS) (Jacks...,...,Detroit,Pistons,DET,0,0,NaN,NaN,NaN,NaN,NaN
67,21500001,77,1,101,1,8:26 PM,4:23,NaN,NaN,Jackson 9' Driving Floating Jump Shot (5 PTS) ...,...,Detroit,Pistons,DET,0,0,NaN,NaN,NaN,NaN,NaN
70,21500001,81,1,98,1,8:27 PM,4:13,Schroder 1' Cutting Layup Shot (2 PTS) (Millsa...,NaN,NaN,...,Atlanta,Hawks,ATL,0,0,NaN,NaN,NaN,NaN,NaN


In [61]:
# for each assist event ....
print("Starting unparsed pass dataset: "+ str(len(AST_df)))
CleanData_df = pd.DataFrame(columns=['SportVU eventNum', 'time ball passed', 'time ball caught', 'passInHull', 'receiverinHull', 'receiverIsOpen' ,'SportVUMomentIndex', 'reversedIndexBallCaught', 'PBPTime' , 'PBP_HOMEDESCRIPTION' , 'PBP_VISITORDESCRIPTION' ])

if(not curPath.endswith('SportVU_JSON_Data')):
    curPath = os.getcwd()
    print curPath
    strIndex=curPath.index('\\JupyterNotebooks\\')
    curPath = curPath[:(strIndex+18)]+'SportVU_JSON_Data\\'
    os.chdir(curPath)
print curPath
#SportVU_JSON_Rootpath = "C:\Users\jchin\Documents\GitHub\Data Mining Course\DataMining-Project-Folder\JupyterNotebooks\SportVU_JSON_Data\\"
#print SportVU_JSON_Rootpath+str(gameid)+".json"
gameid_prev = 0;
for index, ast in AST_df.iterrows():
    gameid = ast["GAME_ID"]    #game id from PBP
    eventNumPBP = ast["EVENTNUM"] #eventNumber for the play
    gametimePBP = ast["PCTIMESTRING"]  #game time for the play
    quarterPBP = int(ast["PERIOD"])
    #getPBPInfo(gameid,eventNumPBP)
    print("prevGameID:"+str(gameid_prev)+", currentlyAnalyzingGameID:"+str(gameid))
    print("PBPEventNum:"+str(eventNumPBP)+", gametimePBP:"+str(gametimePBP))
    #check if SportVU json data is already opened for this PBP play
    if(gameid_prev != gameid):
        #open raw JSON SportVU data using game id from assist
        with open(curPath+"00"+str(gameid)+".json") as data_file:    
            rawSportVUData = json.load(data_file)
            json_normalize(rawSportVUData['events'])
        gameid_prev = gameid
    
    #get the correct eventNum (returned as long) in SportsVU
    eventNumSportsVU = findCorrectMomentData(rawSportVUData, quarterPBP, gametimePBP, eventNumPBP)
    if(eventNumSportsVU is None):
        print("Could not find eventNum in SportsVU")
        continue
    
    
    print("Correct SportVU eventNum: "+str(eventNumSportsVU))
    #extract SportsVU movements for that eventNum
    
    rawpossessionSportsVU = MomentsfromEventNum(rawSportVUData, int(eventNumSportsVU))
    
    #for analyzing AST only...
    receiverID = int(ast['PLAYER1_ID'])
    passerID = int(ast['PLAYER2_ID'])
    teamID = int(ast['PLAYER1_TEAM_ID'])
    #PBPTime = str(ast['PCTIMESTRING'])
    PBP_HOMEDESCRIPTION = str(ast['HOMEDESCRIPTION']) 
    PBP_VISITORDESCRIPTION = str(ast['VISITORDESCRIPTION'])
    
                     
    #get catch details (who was the passer, receiver, and when caught/passed)
    timeBallReleased, indexBallReleased, timeBallCaught, indexBallCaught, timeBallPassed, indexBallPassed, receiverPosAtPass, receiverPosAtCaught, passerPosAtPass , SportVUMomentIndex= getCatchEventDetails(receiverID, passerID, gametimePBP, rawpossessionSportsVU)
    
    if(timeBallReleased is None):
        print("Could not find time for when ball was passed")
        continue
    
    #hull details
    momentsReversed = rawpossessionSportsVU[::-1]
    hull, np_points = getHull(momentsReversed[indexBallCaught], teamID)
                     
    #passer in hull?
    
    passerInHull = isInHull(hull,np_points, passerPosAtPass[0] ,passerPosAtPass[1])
                     
    #receiver in hull?
    receiverInHull = isInHull(hull,np_points, receiverPosAtPass[0] ,receiverPosAtPass[1])
    
    #is receiver open?  ( np_points position of defense)
    closestDist, receiverIsOpen = is_open([receiverPosAtCaught[0],receiverPosAtCaught[1]], np_points)
    
    #PBPTime , PBP_HOMEDESCRIPTION , PBP_VISITORDESCRIPTION = getPBPInfo("00"+str(gameid), eventNumPBP) 
    CleanData_df.loc[len(CleanData_df.index)] = [eventNumSportsVU, timeBallPassed, timeBallCaught, passerInHull, receiverInHull, receiverIsOpen , SportVUMomentIndex , indexBallCaught, gametimePBP , PBP_HOMEDESCRIPTION , PBP_VISITORDESCRIPTION ]


CleanData_df.to_csv("CleanedData.csv", sep='\t')

Starting unparsed pass dataset: 84
D:\DataMining-NBA-Repo\JupyterNotebooks\SportVU_JSON_Data
D:\DataMining-NBA-Repo\JupyterNotebooks\SportVU_JSON_Data\
prevGameID:0, currentlyAnalyzingGameID:21500001
PBPEventNum:5, gametimePBP:11:21
Finding correct SportVU eventNum for gameid:0021500001
681
Currently checking eventNumPBP: 5
sportVU Index for the eventNumPBP: 4
Correct SportVU eventNum: 5
sportVU Index for the eventNumPBP: 4
681
num of moments: 623
starting at time: 681.02
receiver holding ball: 9.62
receiver holding ball: 9.66
receiver holding ball: 9.71
11:23
receiver just caught ball: 12.34 received reversed index219
0:12
receiver just caught ball: 12.38 received reversed index220
0:12
receiver just caught ball: 12.42 received reversed index221
0:12
11:26
passer just passed ball: 12.76 passed reversed index229
passer just passed ball: 12.8 passed reversed index230
player PASSED ball at 12.84
11:26
[[21.95424, 37.78269], [22.02161, 7.38196], [11.36992, 18.75068], [10.1838, 46.8024], [

prevGameID:21500001, currentlyAnalyzingGameID:21500001
PBPEventNum:105, gametimePBP:1:47
Finding correct SportVU eventNum for gameid:0021500001
107
Currently checking eventNumPBP: 105
sportVU Index for the eventNumPBP: 92
Correct SportVU eventNum: 105
sportVU Index for the eventNumPBP: 92
107
num of moments: 425
starting at time: 107.01
receiver holding ball: 11.66
receiver holding ball: 11.7
receiver holding ball: 11.74
1:49
receiver just caught ball: 12.94 received reversed index189
0:12
receiver just caught ball: 12.98 received reversed index190
0:12
receiver just caught ball: 13.02 received reversed index191
0:13
1:50
passer just passed ball: 14.98 passed reversed index240
passer just passed ball: 15.02 passed reversed index241
player PASSED ball at 15.06
1:52
[[75.68995, 24.78592], [89.00367, 6.19983], [70.92586, 29.46816], [59.95027, 14.65658], [78.11457, 39.95304]]
prevGameID:21500001, currentlyAnalyzingGameID:21500001
PBPEventNum:110, gametimePBP:0:32
Finding correct SportVU ev

prevGameID:21500001, currentlyAnalyzingGameID:21500001
PBPEventNum:269, gametimePBP:8:37
Finding correct SportVU eventNum for gameid:0021500001
517
Currently checking eventNumPBP: 269
sportVU Index for the eventNumPBP: 243
Correct SportVU eventNum: 269
sportVU Index for the eventNumPBP: 243
517
num of moments: 514
starting at time: 517.01
receiver holding ball: 8.91
receiver holding ball: 8.95
receiver holding ball: 8.99
8:37
receiver just caught ball: 9.56 received reversed index33
0:9
receiver just caught ball: 9.6 received reversed index34
0:9
receiver just caught ball: 9.64 received reversed index35
0:9
8:37
passer just passed ball: 11.27 passed reversed index75
passer just passed ball: 11.31 passed reversed index76
player PASSED ball at 11.35
8:39
[[64.3194, 28.5193], [64.86101, 34.92905], [82.38713, 42.66825], [70.91426, 12.76705], [77.18193, 10.15939]]
prevGameID:21500001, currentlyAnalyzingGameID:21500001
PBPEventNum:282, gametimePBP:7:42
Finding correct SportVU eventNum for ga

PBPEventNum:364, gametimePBP:11:44
Finding correct SportVU eventNum for gameid:0021500001
704
Currently checking eventNumPBP: 364
sportVU Index for the eventNumPBP: 320
Correct SportVU eventNum: 364
sportVU Index for the eventNumPBP: 320
704
num of moments: 525
starting at time: 704.0
receiver holding ball: 24.0
receiver holding ball: 24.0
receiver holding ball: 24.0
11:44
receiver just caught ball: 24.0 received reversed index138
0:24
receiver just caught ball: 24.0 received reversed index139
0:24
receiver just caught ball: 24.0 received reversed index140
0:24
11:45
passer just passed ball: 11.41 passed reversed index196
passer just passed ball: 11.45 passed reversed index197
player PASSED ball at 11.49
11:47
[[67.30279, 41.1566], [54.09065, 19.5984], [89.73261, 22.31065], [79.8264, 4.3835], [69.33797, 6.26065]]
prevGameID:21500001, currentlyAnalyzingGameID:21500001
PBPEventNum:372, gametimePBP:11:00
Finding correct SportVU eventNum for gameid:0021500001
660
Currently checking eventNu

Finding correct SportVU eventNum for gameid:0021500002
683
Currently checking eventNumPBP: 4
sportVU Index for the eventNumPBP: 3
Correct SportVU eventNum: 4
sportVU Index for the eventNumPBP: 3
683
num of moments: 475
starting at time: 683.02
receiver holding ball: 12.74
receiver holding ball: 12.77
receiver holding ball: 12.82
11:24
receiver just caught ball: 13.57 received reversed index273
0:13
receiver just caught ball: 13.61 received reversed index274
0:13
receiver just caught ball: 13.65 received reversed index275
0:13
11:25
passer just passed ball: 14.24 passed reversed index290
passer just passed ball: 14.28 passed reversed index291
player PASSED ball at 14.32
11:26
[[36.27581, 27.17431], [30.13139, 38.30365], [19.06445, 3.92155], [8.1564, 30.46676], [22.20008, 15.6143]]
prevGameID:21500002, currentlyAnalyzingGameID:21500002
PBPEventNum:24, gametimePBP:9:02
Finding correct SportVU eventNum for gameid:0021500002
542
Currently checking eventNumPBP: 24
sportVU Index for the event

prevGameID:21500002, currentlyAnalyzingGameID:21500002
PBPEventNum:232, gametimePBP:5:17
Finding correct SportVU eventNum for gameid:0021500002
317
Currently checking eventNumPBP: 232
sportVU Index for the eventNumPBP: 202
Correct SportVU eventNum: 232
sportVU Index for the eventNumPBP: 202
317
num of moments: 950
starting at time: 317.02
receiver holding ball: 17.43
receiver holding ball: 17.46
receiver holding ball: 17.5
5:23
receiver just caught ball: 18.28 received reversed index300
0:18
receiver just caught ball: 18.31 received reversed index301
0:18
receiver just caught ball: 18.35 received reversed index302
0:18
5:24
passer just passed ball: 18.88 passed reversed index317
passer just passed ball: 18.91 passed reversed index318
player PASSED ball at 18.95
5:25
[[4.24951, 2.09089], [32.17943, 15.85664], [7.60713, 27.62157], [4.95873, 37.22067], [9.06348, 21.65805]]
prevGameID:21500002, currentlyAnalyzingGameID:21500002
PBPEventNum:251, gametimePBP:3:40
Finding correct SportVU even

prevGameID:21500002, currentlyAnalyzingGameID:21500002
PBPEventNum:346, gametimePBP:5:38
Finding correct SportVU eventNum for gameid:0021500002
338
Currently checking eventNumPBP: 346
sportVU Index for the eventNumPBP: 304
Currently checking eventNumPBP: 347
sportVU Index for the eventNumPBP: 305
Currently checking eventNumPBP: 348
sportVU Index for the eventNumPBP: 306
Could not find eventNum in SportsVU
prevGameID:21500002, currentlyAnalyzingGameID:21500002
PBPEventNum:366, gametimePBP:4:21
Finding correct SportVU eventNum for gameid:0021500002
261
Currently checking eventNumPBP: 366
sportVU Index for the eventNumPBP: 323
Correct SportVU eventNum: 366
sportVU Index for the eventNumPBP: 323
261
num of moments: 425
starting at time: 261.0
receiver holding ball: 14.24
receiver holding ball: 14.28
receiver holding ball: 14.32
4:24
receiver just caught ball: 15.2 received reversed index288
0:15
receiver just caught ball: 15.24 received reversed index289
0:15
receiver just caught ball: 15.

In [52]:
getPBPInfo(21500001,47)

40    6:48
Name: PCTIMESTRING, dtype: object
40    NaN
Name: HOMEDESCRIPTION, dtype: object
40    Ilyasova 24' 3PT Jump Shot (3 PTS) (Marc Morri...
Name: VISITORDESCRIPTION, dtype: object
